<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span><ul class="toc-item"><li><span><a href="#Импортируем-необходимые-библиотеки" data-toc-modified-id="Импортируем-необходимые-библиотеки-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импортируем необходимые библиотеки</a></span></li><li><span><a href="#Загрузим-и-изучим--данные" data-toc-modified-id="Загрузим-и-изучим--данные-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузим и изучим  данные</a></span></li><li><span><a href="#Выводы-по-изучению-данных" data-toc-modified-id="Выводы-по-изучению-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Выводы по изучению данных</a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Выводы-по-подготовке-данных" data-toc-modified-id="Выводы-по-подготовке-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Выводы по подготовке данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Обучение-модели-LogisticRegression" data-toc-modified-id="Обучение-модели-LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Обучение модели LogisticRegression</a></span></li><li><span><a href="#Обучение-модели-DecisionTreeClassifier" data-toc-modified-id="Обучение-модели-DecisionTreeClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение модели DecisionTreeClassifier</a></span></li><li><span><a href="#Обучение-модели-RandomForestClassifier" data-toc-modified-id="Обучение-модели-RandomForestClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Обучение модели RandomForestClassifier</a></span></li><li><span><a href="#Обучение-модели-CatBoostClassifier" data-toc-modified-id="Обучение-модели-CatBoostClassifier-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Обучение модели CatBoostClassifier</a></span></li><li><span><a href="#Сводная-таблица-результатов-обучения-моделей" data-toc-modified-id="Сводная-таблица-результатов-обучения-моделей-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Сводная таблица результатов обучения моделей</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Разработка модели с целью классификации комментариев

Один известный интернет магазин "Х" запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров - кклиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

# Задачи проекта

- Разработать инструмент, который будет автоматически идентифицировать токсичные комментарии, чтобы отправить их на модерацию и обеспечить безопасную и продуктивную среду для редактирования описаний товаров. В распоряжении набор данных с разметкой о токсичности правок.
- Требование заказчика - модель со значением метрики качества *F1* не меньше 0.75. 

# Шаги решения задач проекта

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.


# Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Изучение данных

### Импортируем необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')

### Загрузим и изучим  данные 

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', encoding='utf8')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
# Посмотрим соотношение данных по целевому признаку
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
# Проверим данные на наличие дубликатов
data.duplicated().sum()

0

In [7]:
# Проверим данные на наличие пропусков
data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

### Выводы по изучению данных
- Выявлены лишние символы в столбце с текстом, требующие дальнейшей обработки;
- По соотношению значений целевого признака, лидируют объекты отрицательные класса - классы несбалансированы;
- Пропуски и дубликаты в данных отсутствуют;
- Столбец `Unnamed: 0` не несет ценной информации, лишь дублирует индексы, в дальнейшем избавимся от него.

## Подготовка

In [8]:
# Удалим столбец 'Unnamed: 0'
data = data.drop(['Unnamed: 0'], axis=1)

In [9]:
# Выполним очистку текстов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = " ".join(text.split())
    return text

# Обновим столбец с очищенными данными 
data['text'] = data['text'].apply(clear_text)

In [10]:
import nltk
nltk.download('averaged_perceptron_tagger')

# Выполним лемматизацию с POS-тэгированием слов
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

# Обновим столбец с лемматизированными данными 
data['text'] = data['text'].apply(lemmatize_text)    

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
data.head()

,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [12]:
# Разделим данные на тренировочную, валидационную и тестовую выборки
X = data.drop(['toxic'], axis=1)
y = data['toxic']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.5, random_state = 1234)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size = 0.5, random_state = 1234)

In [13]:
# Избавимся от стоп-слов
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

# Создадим счетчик, указав в нем стоп-слова
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
X_train = count_tf_idf.fit_transform(X_train['text'].values) 
X_valid = count_tf_idf.transform(X_valid['text'].values) 
X_test = count_tf_idf.transform(X_test['text'].values) 

In [15]:
# Посмотрим на размеры выборок
for i in [X_train, y_train, X_valid, y_valid, X_test, y_test]:
    print(i.shape)

(79646, 99708)
(79646,)
(39823, 99708)
(39823,)
(39823, 99708)
(39823,)


In [16]:
# Выполним балансировку данных, уменьшая размер весов классов 
classes = data['toxic'].value_counts()[0]/data['toxic'].value_counts()[1]
dict_class = {0:1, 1:classes}
# Проверим на логистической регрессии с выводом результата метрики f1
model = LogisticRegression(class_weight=dict_class)
train_ballanced = cross_val_score(model, X_train, y_train, cv=3, scoring='f1').mean()
print(round(train_ballanced,2))

0.75


### Выводы по подготовке данных
- Удалили столбец 'Unnamed: 0';
- Выполнили лемматизацию;
- Избавились от стоп-слов;
- Разделили данные на тренировочную, валидационную и тестовую выборки для дальнейшего обучения;
- Выполнили балансировку данных, уменьшая размер весов классов.

## Обучение

### Обучение модели LogisticRegression

In [17]:
LR_model = LogisticRegression()
param_grid = [{'solver':['liblinear', 'newton-cg'],
                'C':[1, 10],
                'class_weight':[dict_class]}]
LR_GSCV = GridSearchCV(LR_model, param_grid, cv = 2, scoring='f1')
LR_GSCV.fit(X_train, y_train)

LR_best_params = LR_GSCV.best_params_
# Выведем результаты
LR_results = pd.DataFrame(LR_GSCV.cv_results_)
display(LR_results[['params', 'mean_test_score', 'rank_test_score']].sort_values('rank_test_score'))
# Отдельно обозначим значение f1 для лучшей модели 
f1_LR_best = round((LR_results['mean_test_score'].max()),2)
print('f1 с кроссвалидацией:', f1_LR_best)

,params,mean_test_score,rank_test_score
2,"{'C': 10, 'class_weight': {0: 1, 1: 8.84134437...",0.752043,1
3,"{'C': 10, 'class_weight': {0: 1, 1: 8.84134437...",0.752043,1
0,"{'C': 1, 'class_weight': {0: 1, 1: 8.841344371...",0.742156,3
1,"{'C': 1, 'class_weight': {0: 1, 1: 8.841344371...",0.742156,3


f1 с кроссвалидацией: 0.75


In [18]:
# Посмотрим на результат на валидационной выборке
LR_model = LogisticRegression()
LR_model.set_params(**LR_best_params)
LR_model.fit(X_train, y_train)

predict = LR_model.predict(X_valid)
f1_LR_valid = f1_score(y_valid, predict)
print('f1 на валидационной выборке:', f1_LR_valid)

f1 на валидационной выборке: 0.7554008641382621


### Обучение модели DecisionTreeClassifier

In [19]:
DTC_model = DecisionTreeClassifier(random_state = 1234)
param_grid = [{'max_depth':(None, *range(2, 25)),
                'class_weight':[dict_class]}]
DTC_GSCV = GridSearchCV(DTC_model, param_grid, cv = 2, scoring='f1')
DTC_GSCV.fit(X_train, y_train)

DTC_best_params = DTC_GSCV.best_params_
# Выведем результаты
DTC_results = pd.DataFrame(DTC_GSCV.cv_results_)
display(DTC_results[['params', 'mean_test_score', 'rank_test_score']].sort_values('rank_test_score'))
# Отдельно обозначим значение f1 для лучшей модели 
f1_DTC_best = round((DTC_results['mean_test_score'].max()),2)
print('f1 с кроссвалидацией:', f1_DTC_best)

,params,mean_test_score,rank_test_score
22,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.622218,1
21,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.622115,2
20,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.620067,3
23,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.619999,4
0,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.616068,5
17,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.615825,6
16,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.614803,7
19,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.614165,8
18,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.613019,9
15,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.611052,10


f1 с кроссвалидацией: 0.62


In [20]:
# Посмотрим на результат на валидационной выборке
DTC_model = DecisionTreeClassifier(random_state = 1234)
DTC_model.set_params(**DTC_best_params)
DTC_model.fit(X_train, y_train)

predict = DTC_model.predict(X_valid)
f1_DTC_valid = f1_score(y_valid, predict)
print('f1 на валидационной выборке:', f1_DTC_valid)

f1 на валидационной выборке: 0.6123260437375746


### Обучение модели RandomForestClassifier

In [21]:
RFC_model = RandomForestClassifier(random_state = 1234)
param_grid = [{'max_depth': (None, *range(2, 20)),
               'criterion': (['entropy']),
               'class_weight': [dict_class]}]
RFC_GSCV = GridSearchCV(RFC_model, param_grid, cv = 2, scoring='f1')
RFC_GSCV.fit(X_train, y_train)

RFC_best_params = RFC_GSCV.best_params_
# Выведем результаты
RFC_results = pd.DataFrame(RFC_GSCV.cv_results_)
display(RFC_results[['params', 'mean_test_score', 'rank_test_score']].sort_values('rank_test_score'))
# Отдельно обозначим значение f1 для лучшей модели 
f1_RFC_best = round((RFC_results['mean_test_score'].max()),2)
print('f1 с кроссвалидацией:', f1_RFC_best)

,params,mean_test_score,rank_test_score
0,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.590628,1
18,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.394673,2
17,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.391621,3
16,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.387024,4
15,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.386163,5
14,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.379654,6
13,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.375178,7
12,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.372423,8
11,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.369843,9
10,"{'class_weight': {0: 1, 1: 8.841344371679229},...",0.364033,10


f1 с кроссвалидацией: 0.59


In [22]:
# Посмотрим на результат на валидационной выборке
RFC_model = RandomForestClassifier(random_state = 1234)
RFC_model.set_params(**RFC_best_params)
RFC_model.fit(X_train, y_train)

predict = RFC_model.predict(X_valid)
f1_RFC_valid = f1_score(y_valid, predict)
print('f1 на валидационной выборке:', f1_RFC_valid)

f1 на валидационной выборке: 0.6135593220338983


### Обучение модели CatBoostClassifier

In [23]:
CBR_model = CatBoostClassifier(random_state = 1234)
param_grid = [{'verbose': ([False]),
               'iterations': ([200])}]
CBR_GSCV = GridSearchCV(CBR_model, param_grid, cv = 2, scoring='f1')
CBR_GSCV.fit(X_train, y_train)

CBR_best_params = CBR_GSCV.best_params_
# Выведем результаты
CBR_results = pd.DataFrame(CBR_GSCV.cv_results_)
display(CBR_results[['params', 'mean_test_score', 'rank_test_score']].sort_values('rank_test_score'))
# Отдельно обозначим значение f1 для лучшей модели 
f1_CBR_best = round((CBR_results['mean_test_score'].max()),2)
print('f1 с кроссвалидацией:', f1_CBR_best)

,params,mean_test_score,rank_test_score
0,"{'iterations': 200, 'verbose': False}",0.717673,1


f1 с кроссвалидацией: 0.72


In [24]:
# Посмотрим на результат на валидационной выборке
CBR_model = CatBoostClassifier(random_state = 1234)
CBR_model.set_params(**CBR_best_params)
CBR_model.fit(X_train, y_train)

predict = CBR_model.predict(X_valid)
f1_CBR_valid = f1_score(y_valid, predict)
print('f1 на валидационной выборке:', f1_CBR_valid)

f1 на валидационной выборке: 0.731605600238308


### Сводная таблица результатов обучения моделей

In [25]:
# Создадим сводную таблицу с результатами получившихся моделей
model = ['LogisticRegression',
         'DecisionTreeClassifier',
         'RandomForestClassifier',
         'CatBoostClassifier'] 
df = {'f1 для лучшей модели на CV': [f1_LR_best,
                                     f1_DTC_best,
                                     f1_RFC_best,
                                     f1_CBR_best],
      'f1 для лучшей модели валидации': [f1_LR_valid,
                                         f1_DTC_valid,
                                         f1_RFC_valid,
                                         f1_CBR_valid]}
result = pd.DataFrame(df, index = model)
display(result)

,f1 для лучшей модели на CV,f1 для лучшей модели валидации
LogisticRegression,0.75,0.755401
DecisionTreeClassifier,0.62,0.612326
RandomForestClassifier,0.59,0.613559
CatBoostClassifier,0.72,0.731606


**Выводы по обучению моделей**
- По сводной таблице, стоит отметить, что требование: "Построить модель со значением метрики качества F1 не меньше 0.75" выполняется у модели **LogisticRegression**.

- Выполним предсказание на тестовых данных, используя модель с лучшими показателями метрики качества F1 - LogisticRegression.

### Тестирование

На данном этапе требуется проверить данные на тестовой выборке и сделать выводы.

In [26]:
LR_GSCV.fit(X_train, y_train)
pred_LR = LR_GSCV.predict(X_test)
f1_LR = f1_score(y_test, pred_LR)
print('f1 на лучшей модели LogisticRegressionr', round(f1_LR,2))

f1 на лучшей модели LogisticRegressionr 0.75


## Выводы

**Выводы по изучению данных:**
- Выявлены лишние символы в столбце с текстом, требующие дальнейшей обработки;
- По соотношению значений целевого признака, лидируют объекты отрицательные класса - классы несбалансированы;
- Пропуски и дубликаты в данных отсутствуют;
- Столбец `Unnamed: 0` не несет ценной информации, лишь дублирует индексы, в дальнейшем избавимся от него.

**Выводы по подготовке данных:**
- Удалили столбец 'Unnamed: 0';
- Выполнили лемматизацию;
- Избавились от стоп-слов;
- Разделили данные на тренировочную, валидационную и тестовую выборки для дальнейшего обучения;
- Выполнили балансировку данных, уменьшая размер весов классов.

**Выводы по обучению моделей:**
- По сводной таблице, стоит отметить, что требование: "Построить модель со значением метрики качества F1 не меньше 0.75" выполняется у модели **LogisticRegression**.
- Выполнили предсказание на тестовых данных, используя модель с лучшими показателями метрики качества F1 - LogisticRegression. Получили значение метрики качества F1 не меньше 0.75.